In [1]:
%pylab qt5

import os
import numpy as np
import glob
import json

from bgan_util import SynthDataset
from bgan_synth import bgan_synth

font = {#'family' : 'normal',
        #'weight' : 'bold',
        'size'   : 28}

matplotlib.rc('font', **font)

Populating the interactive namespace from numpy and matplotlib


/home/hehaodele/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x_dim = 100
z_dim = 10
numz = 10
results_path = "/tmp/bgan_synth"

synth_d = SynthDataset(x_dim)

bresults = bgan_synth(synth_d, z_dim, num_iter=1000, 
                     numz=numz, wasserstein=False, 
                     rpath=results_path,
                     save_weights=True)

(10000, 100)


ValueError: Variable generator/g_h0_lin_W_0000_0000 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/hehaodele/repos/bayesgan.tf/bgan_models.py", line 115, in build_bgan_graph
    shape, initializer=tf.random_normal_initializer(stddev=0.02))
  File "/home/hehaodele/repos/bayesgan.tf/bgan_models.py", line 53, in __init__
    self.build_bgan_graph()
  File "/home/hehaodele/repos/bayesgan.tf/bgan_synth.py", line 89, in bgan_synth
    wasserstein=wasserstein,  # unsupervised only


In [5]:
# ML DCGAN
results = bgan_synth(synth_d, z_dim, num_iter=1000, 
                     numz=1, wasserstein=False, 
                     rpath=results_path,
                     save_weights=True)

ValueError: Variable generator/g_h0_lin_W_0000_0000 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/hehaodele/repos/bayesgan.tf/bgan_models.py", line 115, in build_bgan_graph
    shape, initializer=tf.random_normal_initializer(stddev=0.02))
  File "/home/hehaodele/repos/bayesgan.tf/bgan_models.py", line 53, in __init__
    self.build_bgan_graph()
  File "/home/hehaodele/repos/bayesgan.tf/bgan_synth.py", line 89, in bgan_synth
    wasserstein=wasserstein,  # unsupervised only


In [ ]:
figure()
plot(np.arange(100, 1001, 100), results["divergences"], '--k', linewidth=2, label="ML GAN")
plot(np.arange(100, 1001, 100), bresults["divergences"], '-g', linewidth=2, label="Bayes GAN")
ylim([0.0, 0.6])
title("JS div. (D = %i)" % x_dim)
legend(loc='upper right')
xlabel("No. of iterations")
ylabel("JS Divergence")

In [ ]:
all_wgt_files = []
for it in range(5, 10):
    all_wgt_files.append("%s/weights_%i99.npz" % (results_dir, it))
wgts = np.load(wgtf)

wgt_vec = [list() for _ in xrange(10)]
for name, wgt in wgts.iteritems():
    if "generator" in name:
        gi = int(name[:-2].split("_")[-2])
        wgt_vec[gi].append(wgt.flatten())

In [ ]:
all_wgt_vecs = []
for wgtf in all_wgt_files:
    try:
        wgts = np.load(wgtf)
    except:
        continue
    wgt_vec = [list() for _ in xrange(10)]
    for name, wgt in wgts.iteritems():
        if "generator" in name:
            gi = int(name[:-2].split("_")[-2])
            wgt_vec[gi].append(wgt.flatten())

    for gi in xrange(10):
        wgt_v = np.concatenate(wgt_vec[gi])
        all_wgt_vecs.append(wgt_v[:, None])
    
all_wgts = np.concatenate(all_wgt_vecs, axis=1)

print all_wgts.shape

In [ ]:
from sklearn.manifold import MDS
seed = np.random.RandomState(seed=3)

mds = MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
          dissimilarity="euclidean", metric=True)
pos = mds.fit(all_wgts.T).embedding_

In [ ]:
figure()

plot(pos[:, 0], pos[:, 1], '.')

title("Samples from $p(\\theta_{g} | \mathcal{D})$ in 2D using MDS")
axis("equal")

In [ ]:
#results = bresults

data_real = results["data_real"]
data_fake = results["data_fake"][-1]
aics_fake = results["all_aics_fake"]

print data_real.shape
print type(data_fake)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

data_trans_real = pca.fit_transform(data_real)

print pca.explained_variance_ratio_

print data_trans_real.shape

In [ ]:
#pca = PCA(n_components=2)

data_trans_fake = pca.transform(data_fake)
print data_trans_fake.shape

In [ ]:
figure()
plot(data_trans_real[:, 0], data_trans_real[:, 1], '.r')
plot(data_trans_fake[:, 0], data_trans_fake[:, 1], '*g')